In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import datetime


spark = SparkSession.builder.master("local").appName("test").getOrCreate()


In [3]:
from pyspark.sql.types import *

schema = StructType([
    StructField("id", StringType(), True),
    StructField("txid", StringType(), True),
    StructField("vout_txid", StringType(), True),
    StructField("vout_n", IntegerType(), True),
    StructField("address", StringType(), True),
    StructField("value", FloatType(), True),
    StructField("blockhash", StringType(), True),
    StructField("blocktime", StringType(), True),
    StructField("vout_type", StringType(), True),
    StructField("vout_script", StringType(), True),
    StructField("tx_index", StringType(), True),
    StructField("block_reward", StringType(), True)])

df_raw_out = spark.read.load("gs://bucket-1-btc/*-txOut.csv", format="csv", delimiter="\t", header=False, schema=schema)
df_raw_out.take(1)

[Row(id=None, txid=None, vout_txid=None, vout_n=None, address=None, value=None, blockhash=None, blocktime=None, vout_type=None, vout_script=None, tx_index=None, block_reward=None)]

In [4]:
#sum transation,總交易數
df_raw_out.createOrReplaceTempView("table1")
dis_txid = spark.sql("SELECT distinct txid from table1")
#dis_txid.collect()

dis_txid.count()

18866209

In [5]:
#sum transation,總交易數
#用RDD方式
rdd = df_raw_out.select("txid").rdd
rdd.distinct().count()

18866209

In [6]:
#one block max transation,一個block最大交易數
bloc_txid = spark.sql("select blockhash, count(txid) as txidnum from "+
                      "(SELECT distinct blockhash, txid from table1) as table2 " + 
                      "group by blockhash order by count(txid) DESC")
#bloc_txid.collect()

#bloc_txid.show()
bloc_txid.head(1)

[Row(blockhash='000000000000000000084b32d266e4a728eac41e46e36891712e31ccf78d0485', txidnum=3824)]

In [7]:
#total value,總交易金額
txidcount = spark.sql("select sum(value) from table1")
#txidcount.collect()

txidcount.head(1)

[Row(sum(value)=94755445.49863167)]

In [8]:
#one block max total value,一個block最大交易金額
bloc_totalvalue = spark.sql("SELECT blockhash, sum(value) as totalvalue from table1 " + 
                      "group by blockhash order by sum(value) DESC")
#bloc_totalvalue.collect()

#bloc_txid.show()
bloc_totalvalue.head(1)

[Row(blockhash='000000000000000000133d8b3b6425d4bf686aad50a727a9ee351972828d5c59', totalvalue=155929.94302857365)]